In [319]:
import pandas as pd
from path import Path

import requests
import pprint
import requests
from requests.exceptions import HTTPError
from datetime import datetime as dt
import pandas as pd
import numpy as np
import timeit
import time
import datetime
import copy

import numpy as np
from sklearn import preprocessing
%matplotlib inline

In [320]:
track_cmc_merged_df = pd.read_csv(Path(f"../../../../data/cleandata/track_cmc_merged_df.csv"))

In [321]:
ticker_sentiments_df = pd.read_csv(Path(f"../../../../data/cleandata/ticker_sentiments.csv"))

In [322]:
ticker_sentiments_df.rename(columns = {'ticker':'Ticker'}, inplace = True)

In [323]:
merged_inner = pd.merge(left=track_cmc_merged_df,right=ticker_sentiments_df, left_on='Ticker', right_on='Ticker')

In [324]:
(datetime.datetime.now() - datetime.datetime.strptime(merged_inner.loc[1, "End"][:10], '%Y-%m-%d')).days

848

In [299]:
# merged_inner.drop(merged_inner.loc[merged_inner['Ticker']=='ALGO'].index, inplace = True)

In [325]:
# merged_inner.drop(merged_inner.loc[merged_inner['Ticker']=='AIB'].index, inplace = True)

In [326]:
merged_inner.reset_index(inplace=True)

In [327]:
merged_inner.drop(columns = ['index'], inplace = True)

In [328]:
merged_inner.shape

(573, 26)

In [329]:
merged_inner['Longevity'] = 0
merged_inner['custom_index'] = 0

In [332]:
def custom_index_success(df):
    
    for i in range(len(df)) : 
        # get the longevity of the ICO
        longevity = (datetime.datetime.now() - datetime.datetime.strptime(df.loc[i, "End"][:10], '%Y-%m-%d')).days
        # return to a column for df
        df.loc[i, "Longevity"] = longevity
        # get current price of df, remove commas
        current_price = df.loc[i, "price"].replace(',', '')
        # create custom index
        custom = float(current_price) / (1/(int(str(longevity))))
        # retrun to df
        df.loc[i, "custom_index"] = custom
    return df

In [334]:
merged_df = custom_index_success(merged_inner)
merged_df.head()

,rank_cnc,Name,Ticker,market_cap,price,volume24,pct1h,pct24h,pct7d,country,...,Start,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts,Longevity,custom_index
0,2,Ethereum,ETH,"20,155,411,771",185.67,"7,906,136,399",-0.10,-1.20,-0.96,Switzerland,...,2017-09-28 03:00:00,0 days 00:00:00.000000000,0 days 00:00:00.000000000,0.460570,0.061160,0.027390,0.901510,100,779,144636.930000
1,7,Binance Coin,BNB,"3,277,372,658",21.07,"231,846,713",-0.46,-1.33,2.49,Japan,...,2017-07-01 07:00:00,20 days 00:00:00.000000000,0 days 00:00:00.000000000,0.563773,0.096670,0.041410,0.861890,100,848,17867.360000
2,8,EOS,EOS,"3,214,893,394",3.42,"1,735,988,258",0.11,-1.49,-2.55,Cayman Islands,...,2017-06-26 16:00:00,350 days 08:00:00.000000000,0 days 00:00:00.000000000,0.828470,0.032200,0.858550,0.109270,100,522,1785.240000
3,13,Cardano,ADA,"1,105,888,653",0.042654,"53,437,224",-0.50,-1.72,-2.12,Switzerland,...,2015-09-01 00:00:00,488 days 00:00:00.000000000,0 days 00:00:00.000000000,0.589474,0.098387,0.032946,0.868753,93,1049,44.744046
4,16,Huobi Token,HT,"913,746,412",3.79,"252,881,495",-0.27,-1.89,-4.79,Seychelles,...,2017-09-28 03:00:00,0 days 00:00:00.000000000,0 days 00:00:00.000000000,0.525717,0.081630,0.033480,0.884940,100,779,2952.410000


# using min max scaler

In [347]:
#min max scaler not doing much as the min value is practically 0... thus its taking all the values and dividing it by the max value
list1 = merged_df['custom_index'] - merged_df['custom_index'].min()
list2 = (list1)/(list1.max())
list2

0      1.291616e-01
1      1.595565e-02
2      1.594229e-03
3      3.995667e-05
4      2.636519e-03
5      9.467646e-04
6      3.399190e-03
7      4.804865e-02
8      3.311301e-03
9      2.055215e-04
10     1.601390e-02
11     2.026944e-03
12     2.377592e-04
13     2.096889e-04
14     4.906213e-07
15     7.900957e-04
16     1.501868e-02
17     2.294540e-05
18     7.980025e-05
19     8.811699e-04
20     5.390471e-04
21     1.985679e-04
22     7.706617e-07
23     2.800292e-07
24     1.080065e-04
25     8.677370e-04
26     1.839788e-05
27     2.573745e-07
28     1.849497e-04
29     3.367239e-03
           ...     
543    3.747480e-05
544    6.911405e-05
545    4.311387e-08
546    1.516059e-06
547    1.211483e-07
548    4.779871e-07
549    2.843401e-06
550    5.008110e-08
551    3.245000e-07
552    9.424961e-04
553    2.570498e-05
554    6.197058e-08
555    1.551710e-06
556    4.207576e-06
557    1.299460e-06
558    1.020183e-07
559    4.870222e-06
560    1.196355e-07
561    3.039832e-07


In [348]:
merged_df['custom_index'].min()

1.8502e-05

In [349]:
scaler = preprocessing.MinMaxScaler()

In [350]:
list3 = scaler.fit_transform(merged_df[['custom_index']])

In [360]:
list2.to_list() == list3.tolist

False

In [361]:
merged_df['custom_index_scaled'] = list2

In [373]:
merged_df['custom_index_scaled'].mean()

0.0023699774699249844

In [418]:
def success_of_ico(df):
    mean = df['custom_index_scaled'].mean()
    
    for i in range(len(df)) : 
        if df.loc[i, 'custom_index_scaled'] >= mean:
            df.loc[i, 'success'] = 1
        
        else:
            df.loc[i, 'success'] = 0
            
    return df

In [419]:
merged_df['success'] = 0
success_df = success_of_ico(merged_df)

In [445]:
success_df.drop(columns = ['Name_y'], inplace = True)

In [446]:
file_path = Path("../../../../data/cleandata/success_df.csv")
success_df.to_csv(file_path, index=False)